# Gradio Day!

Today we will build User Interfaces using the outrageously simple Gradio framework.

Prepare for joy!

Please note: your Gradio screens may appear in 'dark mode' or 'light mode' depending on your computer settings.

In [1]:
# imports

import os
import requests
from bs4 import BeautifulSoup
from typing import List
from dotenv import load_dotenv
from openai import OpenAI
#import google.generativeai
#import anthropic


In [6]:
import gradio as gr # oh yeah!

In [17]:
# Connect to OpenAI, Anthropic and Google; comment out the Claude or Google lines if you're not using them

# openai = OpenAI()

# claude = anthropic.Anthropic()

# google.generativeai.configure()

load_dotenv(override=True)

api_key = os.getenv('Open_Router_Key')
if api_key and api_key.startswith('sk-or-v1') and len(api_key)>10:
   print("API key looks good so far")
else:
   print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
  
  
openai = OpenAI(
    api_key=api_key,
    base_url="https://openrouter.ai/api/v1"
)

MODEL_Gemini2FlashLite = 'google/gemini-2.0-flash-lite-preview-02-05:free'
MODEL_Gemini2FlashThink = 'google/gemini-2.0-flash-thinking-exp:free'
MODEL_Gemini2Pro ='google/gemini-2.0-pro-exp-02-05:free'
MODEL_Meta_Llama33 ='meta-llama/llama-3.3-70b-instruct:free'
MODEL_Deepseek_V3='deepseek/deepseek-chat:free'
MODEL_Deepseek_R1='deepseek/deepseek-r1-distill-llama-70b:free'
MODEL_Qwen_vlplus='qwen/qwen-vl-plus:free'
MODEL_OpenAi_o3mini = 'openai/o3-mini'
MODEL_OpenAi_4o = 'openai/gpt-4o-2024-11-20'
MODEL_Claude_Haiku = 'anthropic/claude-3.5-haiku-20241022'


Default_Model = MODEL_Deepseek_V3



API key looks good so far


In [3]:
# A generic system message - no more snarky adversarial AIs!

system_message = "You are a helpful assistant"

In [4]:
# Let's wrap a call to GPT-4o-mini in a simple function

def message_gpt(prompt):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
      ]
    completion = openai.chat.completions.create(
        model=Default_Model,
        messages=messages,
    )
    return completion.choices[0].message.content

In [7]:
# This can reveal the "training cut off", or the most recent date in the training data

message_gpt("What is today's date?")

'Today is October 26, 2023.\n'

## User Interface time!

In [8]:
# here's a simple function

def shout(text):
    print(f"Shout has been called with input {text}")
    return text.upper()

In [9]:
shout("hello")

Shout has been called with input hello


'HELLO'

In [ ]:
# The simplicty of gradio. This might appear in "light mode" - I'll show you how to make this in dark mode later.

gr.Interface(fn=shout, inputs="textbox", outputs="textbox").launch()

In [ ]:
# Adding share=True means that it can be accessed publically
# A more permanent hosting is available using a platform called Spaces from HuggingFace, which we will touch on next week
# NOTE: Some Anti-virus software and Corporate Firewalls might not like you using share=True. If you're at work on on a work network, I suggest skip this test.

gr.Interface(fn=shout, inputs="textbox", outputs="textbox", flagging_mode="never").launch(share=True)

In [ ]:
# Adding inbrowser=True opens up a new browser window automatically

gr.Interface(fn=shout, inputs="textbox", outputs="textbox", flagging_mode="never").launch(inbrowser=True)

## Forcing dark mode

Gradio appears in light mode or dark mode depending on the settings of the browser and computer. There is a way to force gradio to appear in dark mode, but Gradio recommends against this as it should be a user preference (particularly for accessibility reasons). But if you wish to force dark mode for your screens, below is how to do it.

In [ ]:
# Define this variable and then pass js=force_dark_mode when creating the Interface

force_dark_mode = """
function refresh() {
    const url = new URL(window.location);
    if (url.searchParams.get('__theme') !== 'dark') {
        url.searchParams.set('__theme', 'dark');
        window.location.href = url.href;
    }
}
"""
gr.Interface(fn=shout, inputs="textbox", outputs="textbox", flagging_mode="never", js=force_dark_mode).launch()

In [ ]:
# Inputs and Outputs

view = gr.Interface(
    fn=shout,
    inputs=[gr.Textbox(label="Your message:", lines=6)],
    outputs=[gr.Textbox(label="Response:", lines=8)],
    flagging_mode="never"
)
view.launch()

In [ ]:
# And now - changing the function from "shout" to "message_gpt"

view = gr.Interface(
    fn=message_gpt,
    inputs=[gr.Textbox(label="Your message:", lines=6)],
    outputs=[gr.Textbox(label="Response:", lines=8)],
    flagging_mode="never"
)
view.launch()

In [ ]:
# Let's use Markdown
# Are you wondering why it makes any difference to set system_message when it's not referred to in the code below it?
# I'm taking advantage of system_message being a global variable, used back in the message_gpt function (go take a look)
# Not a great software engineering practice, but quite sommon during Jupyter Lab R&D!

system_message = "You are a helpful assistant that responds in markdown"

view = gr.Interface(
    fn=message_gpt,
    inputs=[gr.Textbox(label="Your message:")],
    outputs=[gr.Markdown(label="Response:")],
    flagging_mode="never"
)
view.launch()

In [15]:
# Let's create a call that streams back results
# If you'd like a refresher on Generators (the "yield" keyword),
# Please take a look at the Intermediate Python notebook in week1 folder.

def stream_gpt(prompt):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
      ]
    stream = openai.chat.completions.create(
        model=Default_Model,
        messages=messages,
        stream=True
    )
    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result

In [ ]:
view = gr.Interface(
    fn=stream_gpt,
    inputs=[gr.Textbox(label="Your message:")],
    outputs=[gr.Markdown(label="Response:")],
    flagging_mode="never"
)
view.launch()

In [ ]:
# def stream_claude(prompt):
#     result = claude.messages.stream(
#         model="claude-3-haiku-20240307",
#         max_tokens=1000,
#         temperature=0.7,
#         system=system_message,
#         messages=[
#             {"role": "user", "content": prompt},
#         ],
#     )
#     response = ""
#     with result as stream:
#         for text in stream.text_stream:
#             response += text or ""
#             yield response

In [ ]:
Default_Model=MODEL_Claude_Haiku
view = gr.Interface(
    fn=stream_gpt,
    inputs=[gr.Textbox(label="Your message:")],
    outputs=[gr.Markdown(label="Response:")],
    flagging_mode="never"
)
view.launch()

## Minor improvement

I've made a small improvement to this code.

Previously, it had these lines:

```
for chunk in result:
  yield chunk
```

There's actually a more elegant way to achieve this (which Python people might call more 'Pythonic'):

`yield from result`

I cover this in more detail in the Intermediate Python notebook in the week1 folder - take a look if you'd like more.

In [20]:
def stream_model(prompt, model):
    if model=="GPT":
        Default_Model=MODEL_Gemini2FlashThink
        result = stream_gpt(prompt)
    elif model=="Claude":
        Default_Model=MODEL_Claude_Haiku
        result = stream_gpt(prompt)
    else:
        raise ValueError("Unknown model")
    yield from result

In [ ]:
view = gr.Interface(
    fn=stream_model,
    inputs=[gr.Textbox(label="Your message:"), gr.Dropdown(["GPT", "Claude"], label="Select model", value="GPT")],
    outputs=[gr.Markdown(label="Response:")],
    flagging_mode="never"
)
view.launch()

# Building a company brochure generator

Now you know how - it's simple!

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you read the next few cells</h2>
            <span style="color:#900;">
                Try to do this yourself - go back to the company brochure in week1, day5 and add a Gradio UI to the end. Then come and look at the solution.
            </span>
        </td>
    </tr>
</table>

In [ ]:
# A class to represent a Webpage

class Website:
    url: str
    title: str
    text: str

    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [ ]:
# With massive thanks to Bill G. who noticed that a prior version of this had a bug! Now fixed.

system_message = "You are an assistant that analyzes the contents of a company website landing page \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown."

In [ ]:
def stream_brochure(company_name, url, model):
    prompt = f"Please generate a company brochure for {company_name}. Here is their landing page:\n"
    prompt += Website(url).get_contents()
    if model=="GPT":
        result = stream_gpt(prompt)
    elif model=="Claude":
        result = stream_claude(prompt)
    else:
        raise ValueError("Unknown model")
    yield from result

In [ ]:
view = gr.Interface(
    fn=stream_brochure,
    inputs=[
        gr.Textbox(label="Company name:"),
        gr.Textbox(label="Landing page URL including http:// or https://"),
        gr.Dropdown(["GPT", "Claude"], label="Select model")],
    outputs=[gr.Markdown(label="Brochure:")],
    flagging_mode="never"
)
view.launch()